In [171]:
from collections import defaultdict
from pathlib import Path
import nltk 
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.decomposition import PCA
import re
import pymorphy2
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm_notebook as tqdm
import gensim
import logging
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
import json
from nltk.corpus import stopwords
import warnings

In [422]:
idfiles = [2561, 2049, 4, 2565, 2562, 2062, 3600, 3603, 3588, 534, 3607, 24, 25, 3612, 3102, 2591, 3804, 3627, 1580, 46, 1074, 2100, 3125, 56, 1593, 2485, 2621, 63, 3139, 2316, 610, 2121, 1098, 1099, 588, 2126, 2639, 88, 2140, 607, 1634, 2067, 612, 3175, 3178, 531, 1647, 1136, 1137, 1139, 1146, 1151, 22, 1710, 2692, 1160, 1674, 1560, 3729, 623, 3224, 153, 2206, 1183, 160, 3235, 3748, 684, 1651, 1711, 30, 1714, 2079, 695, 2232, 698, 189, 1227, 1025, 711, 3784, 2253, 2763, 3788, 1741, 2255, 1232, 3283, 383, 2775, 2265, 1829, 1244, 3300, 1767, 1257, 1320, 1747, 3830, 1152, 1279, 3030, 913, 3332, 1798, 2311, 386, 1289, 3340, 269, 1806, 274, 1425, 789, 2863, 1305, 2332, 800, 1825, 2339, 292, 293, 2855, 3368, 562, 2861, 3886, 2351, 2864, 307, 2356, 822, 1019, 3722, 2101, 3387, 2688, 829, 3904, 833, 2279, 3398, 2887, 336, 337, 3473, 854, 3420, 2398, 3426, 3939, 3940, 357, 2918, 1387, 2924, 879, 3442, 1396, 2421, 2427, 1407, 1345, 2438, 1928, 1419, 2456, 1054, 2030, 1937, 402, 2628, 2968, 409, 1437, 1956, 421, 1963, 943, 2481, 3509, 955, 2635, 2496, 455, 972, 3025, 1188, 3542, 3544, 3550, 482, 3699, 2812, 2534, 3566, 1521, 3570, 1525, 2043]

text_test = []
text_base = []
for i in idfiles:
    f = open('/data/share/lab05data/test_'+str(i)+".txt", 'r', encoding='utf-8')
    t = f.readlines()
    text_test.append(t)
for j in range(1, 21):
    s = open('/data/share/lab05data/base_'+str(j)+".txt", 'r', encoding='utf-8')
    t2=s.readlines()
    text_base.append(t2)


In [423]:
stops = set(stopwords.words('russian','english'))

In [424]:
# формирование весов 
cv = CountVectorizer(tokenizer=token_r)
mx_tf = cv.fit_transform(base_text['text']).toarray()

In [425]:
# частые слова
words = [y[0] for y in sorted(cv.vocabulary_.items(), key=lambda x: x[1])]
c_words = zip(words, sum(mx_tf))

In [426]:
c_words = sorted(c_words, key=lambda x: x[1], reverse=True)

In [427]:
stop_w = set([x[0] for x in c_words[:6]])
stop_w = list(stop_w | stops)

In [428]:
GROUPING_SPACE_REGEX = re.compile(r'([^\w]|[+])', re.UNICODE)
m = pymorphy2.MorphAnalyzer()
def simple_word_tokenize(text, _split=GROUPING_SPACE_REGEX.split):
    return [t for t in _split(text.lower()) if t and not t.isspace()]

def token_r(text):
    words = simple_word_tokenize(text)
    return [m.parse(x)[0].normal_form for x in words if len(x) >= 4]



In [452]:
text_base = [str (item) for item in text_base]
text_test = [str (item) for item in text_test]

In [450]:
cv = CountVectorizer(tokenizer=token_r, stop_words=stop_w)

In [451]:
mx_tf = cv.fit_transform(text_base).toarray()

In [453]:
new_entry = cv.transform(text_test)

In [454]:
cos_m = cosine_similarity( new_entry,mx_tf)

In [458]:
n = 0
mn = np.asarray([i.sum() for i in cos_m]).mean()

defined = []
other = []
for i in cos_m:
    if i.sum()>mn:
        defined.append(idfiles[n])
    else:
        other.append(idfiles[n])    
    n +=1



In [467]:
import json

# создаем словарь x:
x = {
"defined": defined,
"other": other
}
# конвертируем в JSON: 
y = json.dumps(x)
# в результате получаем строк JSON:
y.to_json(orient='records')

{"defined": [2561, 2049, 4, 2565, 3600, 3603, 3588, 534, 3607, 24, 3612, 3804, 3627, 2621, 3139, 2316, 1098, 1099, 2639, 88, 2140, 607, 531, 1647, 1136, 1137, 1139, 22, 2692, 1160, 1674, 3224, 153, 3748, 684, 30, 3784, 2763, 1741, 3283, 1244, 1767, 1747, 3830, 1152, 1279, 3030, 3332, 2311, 3340, 789, 2863, 2332, 800, 1825, 292, 2855, 3368, 2861, 3886, 2351, 2864, 822, 3387, 829, 3904, 833, 336, 3473, 854, 3426, 3940, 357, 2918, 1387, 2924, 879, 3442, 2421, 2427, 1407, 1928, 2030, 1937, 402, 2628, 421, 1963, 955, 455, 972, 1188, 3544, 2534, 3566, 2043], "other": [2562, 2062, 25, 3102, 2591, 1580, 46, 1074, 2100, 3125, 56, 1593, 2485, 63, 610, 2121, 588, 2126, 1634, 2067, 612, 3175, 3178, 1146, 1151, 1710, 1560, 3729, 623, 2206, 1183, 160, 3235, 1651, 1711, 1714, 2079, 695, 2232, 698, 189, 1227, 1025, 711, 2253, 3788, 2255, 1232, 383, 2775, 2265, 1829, 3300, 1257, 1320, 913, 1798, 386, 1289, 269, 1806, 274, 1425, 1305, 2339, 293, 562, 307, 2356, 1019, 3722, 2101, 2688, 2279, 3398, 2887, 

In [471]:
with open("lab05.json", "w") as write_file:
    json.dump(y, write_file)